In [ ]:
!pip install fastapi uvicorn google-api-python-client google-auth-oauthlib google-auth-httplib2 pyngrok

# Bibliotecas
import os
import json
import logging
import threading
import base64
import logging
from pathlib import Path
from datetime import datetime
from typing import List, Optional
from logging.handlers import RotatingFileHandler
import sys

# Google APIs
from google.oauth2 import service_account
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from googleapiclient.errors import HttpError

# FastAPI
from fastapi import FastAPI, BackgroundTasks, HTTPException
from fastapi.responses import JSONResponse
from pydantic import BaseModel

# Ngrok e servidor
from pyngrok import ngrok
import nest_asyncio
import uvicorn

In [ ]:

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)  # Configura o nível no logger principal

SCOPES = [
    'https://www.googleapis.com/auth/gmail.readonly',
    'https://www.googleapis.com/auth/drive'
]

#Remove todos os handlers existentes
for handler in logger.handlers[:]:
    logger.removeHandler(handler)

#Adiciona apenas um handler personalizado
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
logger.addHandler(handler)

#Desabilita propagação para handlers de outros módulos
logger.propagate = False

#Constantes
PORT = 8000
CONFIGS_DIR = Path("configs")
TOKENS_DIR = Path("tokens")
NGROK_AUTH_TOKEN = "2yvuP1kc0aUCxRb1wAy4exos2Tp_4dciWNtnLJjErWMUsePeS"

#Cria diretórios necessários
CONFIGS_DIR.mkdir(exist_ok=True)
TOKENS_DIR.mkdir(exist_ok=True)

In [ ]:
# Modelo Pydantic
class AutomacaoConfig(BaseModel):
    email: str
    palavras_chave: List[str]
    destino_nuvem: str
    pasta_destino: str
    renomear: bool
    usar_planilha_log: bool
    link_planilha_log: Optional[str] = None

In [ ]:
# Inicializa FastAPI
app = FastAPI()

In [ ]:
def processar_anexos(service, message: dict, destino_local: str = "downloads") -> List[str]:
    """Processa e salva anexos de uma mensagem do Gmail"""
    try:
        os.makedirs(destino_local, exist_ok=True)
        arquivos = []

        #Verifica se a mensagem tem partes
        if 'parts' not in message['payload']:
            return arquivos

        for part in message['payload']['parts']:
            if part.get('filename') and part['body'].get('attachmentId'):
                try:
                    #Obtém o anexo
                    attachment = service.users().messages().attachments().get(
                        userId='me',
                        messageId=message['id'],
                        id=part['body']['attachmentId']
                    ).execute()

                    #Decodifica e salva o arquivo
                    file_data = base64.urlsafe_b64decode(attachment['data'].encode('UTF-8'))
                    file_path = os.path.join(destino_local, part['filename'])

                    with open(file_path, 'wb') as f:
                        f.write(file_data)

                    arquivos.append(file_path)
                    logger.info(f"Anexo salvo: {file_path}")

                except Exception as e:
                    logger.error(f"Erro ao processar anexo {part.get('filename')}: {str(e)}")
                    continue

        return arquivos

    except Exception as e:
        logger.error(f"Erro geral ao processar anexos: {str(e)}")
        return []

In [ ]:
def escolher_credenciais():
    """
    Se houver tokens em 'tokens/', pede para o usuário escolher.
    Se não houver, executa fluxo OAuth manual para gerar o primeiro token.
    """
    TOKEN_DIR = 'tokens'
    SCOPES = [
        'https://www.googleapis.com/auth/gmail.readonly',
        'https://www.googleapis.com/auth/drive'
    ]

    if not os.path.exists(TOKEN_DIR):
        os.makedirs(TOKEN_DIR)

    tokens = [f for f in os.listdir(TOKEN_DIR) if f.endswith('.json')]

    if not tokens:
        print("Nenhum token encontrado. Iniciando geração de um novo token.")

        email_hint = input("Digite um e-mail para a autorização: ").strip() or None

        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json',
            scopes=SCOPES,
            redirect_uri='urn:ietf:wg:oauth:2.0:oob'
        )

        auth_url, _ = flow.authorization_url(
            access_type='offline',
            prompt='consent',
            login_hint=email_hint
        )

        print("Abra este link para autorizar o acesso:")
        print(auth_url)

        code = input("Cole aqui o código de autorização: ")

        flow.fetch_token(code=code)

        creds = flow.credentials

        email_salvar = input("Digite um nome para o token: ")
        token_path = os.path.join(TOKEN_DIR, f'token_{email_salvar.replace("@", "_at_").replace(".", "_dot_")}.json')

        with open(token_path, 'w') as token_file:
            token_file.write(creds.to_json())

        print(f"Token salvo em: {token_path}")
        return creds

    else:
        print("Tokens disponíveis:")
        for idx, nome in enumerate(tokens, 1):
            print(f"{idx}. {nome}")

        while True:
            try:
                escolha = int(input(f"Escolha um token (1-{len(tokens)}): "))
                if 1 <= escolha <= len(tokens):
                    break
                else:
                    print("Opção inválida. Tente novamente.")
            except ValueError:
                print("Digite um número válido.")

        token_escolhido = os.path.join(TOKEN_DIR, tokens[escolha - 1])
        print(f"Usando token: {token_escolhido}")
        creds = Credentials.from_authorized_user_file(token_escolhido, SCOPES)
        return creds

In [ ]:
credenciais_escolhidas = escolher_credenciais()

Nenhum token encontrado. Iniciando geração de um novo token.
Digite um e-mail para a autorização: leonardo.distante@gmail.com
Abra este link para autorizar o acesso:
https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=812839091248-brlged2alguefhcovkfceo2akrq8gqfv.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=a56xfmiLeKIQ9fVFKhdHtQ9H5SkE9X&access_type=offline&prompt=consent&login_hint=leonardo.distante%40gmail.com
Cole aqui o código de autorização: 4/1AVMBsJgXv9IYZ3QrnsuCSRazvWL5ETaKzCQpPpOxRKEBaJ-NCJw5zkUC06I
Digite um nome para o token: leonardo
Token salvo em: tokens/token_leonardo.json


In [ ]:
def upload_para_drive(caminho_arquivo: str, pasta_id: str) -> str:
    """Faz upload de um arquivo para o Google Drive"""
    try:
        creds = credenciais_escolhidas

        service = build('drive', 'v3', credentials=creds)

        file_metadata = {
            'name': os.path.basename(caminho_arquivo),
            'parents': [pasta_id]
        }

        media = MediaFileUpload(caminho_arquivo)

        file = service.files().create(
            body=file_metadata,
            media_body=media,
            fields='id'
        ).execute()

        logger.info(f"Arquivo {caminho_arquivo} enviado para Drive com ID: {file.get('id')}")
        return file.get('id')

    except Exception as e:
        logger.error(f"Erro no upload para Drive: {str(e)}")
        raise

In [ ]:
@app.post("/api/configurar-automacao")
async def configurar_automacao(config: AutomacaoConfig, background_tasks: BackgroundTasks):
    try:
        # Salva configuração
        config_path = CONFIGS_DIR / f"{config.email}.json"
        with open(config_path, 'w') as f:
            json.dump(config.dict(), f, indent=2)

        # Verifica credenciais (agora usando apenas a variável global)
        if not credenciais_escolhidas:
            raise HTTPException(status_code=401, detail="Nenhuma credencial configurada")

        if not credenciais_escolhidas.valid:
            try:
                credenciais_escolhidas.refresh(Request())
            except Exception as e:
                raise HTTPException(status_code=401, detail=f"Credenciais expiradas e falha ao renovar: {str(e)}")

        service = build('gmail', 'v1', credentials=credenciais_escolhidas)

        #Busca mensagens
        query = f"from:{config.email} OR subject:{' OR subject:'.join(config.palavras_chave)} has:attachment"
        results = service.users().messages().list(
            userId='me',
            q=query,
            maxResults=20
        ).execute()

        messages = results.get('messages', [])

        #Processa mensagens
        arquivos_processados = []
        for msg in messages:
            try:
                full_msg = service.users().messages().get(
                    userId='me',
                    id=msg['id'],
                    format='full'
                ).execute()

                arquivos = processar_anexos(service, full_msg)
                arquivos_processados.extend(arquivos)

                # Envia para o Google Drive
                if config.destino_nuvem.lower() == "google drive":
                    for arquivo in arquivos:
                        background_tasks.add_task(
                            upload_para_drive,
                            arquivo,
                            config.pasta_destino
                        )

            except Exception as e:
                logger.error(f"Erro ao processar mensagem {msg['id']}: {e}")
                continue

        return {
            "status": "success",
            "email": config.email,
            "arquivos_processados": len(arquivos_processados),
            "mensagens_processadas": len(messages)
        }

    except Exception as e:
        logger.error(f"Erro no endpoint: {str(e)}")
        raise HTTPException(status_code=500, detail=str(e))

In [ ]:
@app.get("/api/configs")
async def listar_configs():
    """Lista todas as configurações salvas"""
    configs = []
    for config_file in CONFIGS_DIR.glob("*.json"):
        with open(config_file, 'r') as f:
            configs.append(json.load(f))
    return configs


In [ ]:
if __name__ == "__main__":
    try:
        # Configuração de logging
        logger = logging.getLogger(__name__)
        logger.setLevel(logging.INFO)

        # Remove handlers existentes (evita duplicação)
        for handler in logger.handlers[:]:
            logger.removeHandler(handler)

        # Configura handler personalizado
        handler = logging.StreamHandler()
        handler.setFormatter(logging.Formatter(
            '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
        ))
        logger.addHandler(handler)
        logger.propagate = False

        try:
            current_file = Path(__file__).stem
            app_str = f"{current_file}:app"
        except NameError:
            app_str = "__main__:app"

        # Iniciar servidor Uvicorn em thread separada
        def run_server():
            uvicorn.run(
                app_str,
                host="0.0.0.0",
                port=PORT,
                log_level="info",
                reload=False,
                workers=1
            )

        server_thread = threading.Thread(target=run_server, daemon=True)
        server_thread.start()

        # Esperar o servidor iniciar
        import time
        time.sleep(5)

        # Configurar Ngrok
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        nest_asyncio.apply()

        try:
            # Criar túnel ngrok
            public_url = ngrok.connect(PORT, bind_tls=True).public_url
            logger.info(f"\n{'='*50}")
            logger.info(f"Servidor rodando localmente em: http://localhost:{PORT}")
            logger.info(f"URL pública Ngrok: {public_url}")
            logger.info(f"Documentação Swagger: {public_url}/docs")
            logger.info(f"Documentação Redoc: {public_url}/redoc")
            logger.info(f"{'='*50}\n")
            logger.info("Pressione Ctrl+C para encerrar\n")

            # Manter a aplicação rodando
            while True:
                time.sleep(1)

        except Exception as e:
            logger.error(f"Erro ao conectar Ngrok: {str(e)}")
            raise

    except KeyboardInterrupt:
        logger.info("\n Encerrando servidor")
        ngrok.kill()  # Encerrar todos os túneis
        sys.exit(0)

    except Exception as e:
        logger.critical(f"Erro crítico: {str(e)}")
        ngrok.kill()
        sys.exit(1)

INFO:     Started server process [4181]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


2025-07-18 15:21:42,448 - __main__ - INFO - 
2025-07-18 15:21:42,449 - __main__ - INFO - Servidor rodando localmente em: http://localhost:8000
2025-07-18 15:21:42,450 - __main__ - INFO - URL pública Ngrok: https://6a74cff9c290.ngrok-free.app
2025-07-18 15:21:42,453 - __main__ - INFO - Documentação Swagger: https://6a74cff9c290.ngrok-free.app/docs
2025-07-18 15:21:42,461 - __main__ - INFO - Documentação Redoc: https://6a74cff9c290.ngrok-free.app/redoc
2025-07-18 15:21:42,462 - __main__ - INFO - ==================================================

2025-07-18 15:21:42,464 - __main__ - INFO - Pressione Ctrl+C para encerrar



INFO:     2804:14d:6891:8a91:d595:fe60:57a0:abb0:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2804:14d:6891:8a91:d595:fe60:57a0:abb0:0 - "GET /openapi.json HTTP/1.1" 200 OK


/tmp/ipython-input-9-2897591255.py:7: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  json.dump(config.dict(), f, indent=2)
2025-07-18 15:22:21,757 - __main__ - INFO - Anexo salvo: downloads/35240247960950020157550020000968371055935836.xml
2025-07-18 15:22:22,040 - __main__ - INFO - Anexo salvo: downloads/35240247960950020157550020000968371055935836.pdf
2025-07-18 15:22:22,452 - __main__ - INFO - Anexo salvo: downloads/31241219324150000260550010000163971060684047-nfe.xml
2025-07-18 15:22:22,705 - __main__ - INFO - Anexo salvo: downloads/01e752e598ad4ce5fc1f9b5260db42d6.pdf
2025-07-18 15:22:23,150 - __main__ - INFO - Anexo salvo: downloads/35190958732058000290550160002589491008172648-nfe.xml
2025-07-18 15:22:23,405 - __main__ - INFO - Anexo salvo: downloads/35190958732058000290550160002589491008172648-nfe.pdf
2025-07-18

INFO:     2804:14d:6891:8a91:d595:fe60:57a0:abb0:0 - "POST /api/configurar-automacao HTTP/1.1" 200 OK


2025-07-18 15:22:26,966 - __main__ - INFO - Arquivo downloads/35240247960950020157550020000968371055935836.xml enviado para Drive com ID: 1g1NBF-HU-24BRWSeEZtG2j3tUvk3grDr
2025-07-18 15:22:28,530 - __main__ - INFO - Arquivo downloads/35240247960950020157550020000968371055935836.pdf enviado para Drive com ID: 1f8-p3FlGCWHEfzZhPm4u4fQQGqSs6XWO
2025-07-18 15:22:30,012 - __main__ - INFO - Arquivo downloads/31241219324150000260550010000163971060684047-nfe.xml enviado para Drive com ID: 1Z2RdklZE9535OwU5TOk_nfvUaZUX2j1S
2025-07-18 15:22:31,409 - __main__ - INFO - Arquivo downloads/01e752e598ad4ce5fc1f9b5260db42d6.pdf enviado para Drive com ID: 1Ivy_OhUxju0MkiStjDFVzJYCW3JYxO7_
2025-07-18 15:22:33,099 - __main__ - INFO - Arquivo downloads/35190958732058000290550160002589491008172648-nfe.xml enviado para Drive com ID: 1IpBoPw9y28ml-AmXOLNnt7CtcubJP66b
2025-07-18 15:22:34,634 - __main__ - INFO - Arquivo downloads/35190958732058000290550160002589491008172648-nfe.pdf enviado para Drive com ID: 1-